# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 1

- Проработать проблематику (в чем смысл для бизнеса)
- Грамотно формализовать задачу
- Проанализировать имеющиеся данные и оценить их пригодность для решения поставленной задачи
- Провести первичный разведочный анализ данных (EDA)

# Проблематика

## Тезисы
1. WB &mdash; маркетплейс по продаже различных товаров.
2. Цель бизнеса &mdash; извлечение наибольшей прибыли. 
3. На WB много конкурирующих продавцов.
4. Информация о предпочтениях и будущих покупках пользователей
позволит продавцу заработать на релевантных товарах и не разориться на убыточных.

## Развернутое пояснение тезисов и проблематики

Как правило, продавцы специализируются на одной категории товаров, занимают свою нишу, в которой необходимо быть, если не лучшими и единственными, то одними из лучших для получения значимой прибыли. 
Следовательно, необходим анализ рынка товаров, необходимо знать свою целевую аудиториию и ее потребности, чтобы грамотно распределять средства: вовремя закупать товары, сокращать их поставки или изымать из товарооборота как неликвидные. 

## Задача

Необходимо предоставить инструмент, способный помочь бизнесу в анализе релевантности товаров, дать оценку ликвидности имеющихся изделий на витрине, дать возможность спрогнозировать тренд.

### Фомулировка в терминах ML

Требуется составить рекомендательную ML-модель, которая будет отвечать на вопрос: "Что пользователь купит дальше?"; построить рекомендательную систему, которая основывается на взаимодействии пользователя с товаром и выдает возможные варианты следующих взаимодействий.

# Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import dill

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import seaborn as sns

# USE THIS STYLE
# plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# 
# OR THIS STYLE
import aquarel

import warnings

warnings.filterwarnings("ignore")

theme = aquarel.load_theme("arctic_light")
theme.set_font(family="serif")
theme.apply()

# Сделаем автоподгрузку всех изменений при перепрогонке ячейки
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Данные

Предоставлено три архива с данными. Рассмотрим каждый из них по отдельности и сделаем вывод по каждому архиву данных.


Путь до данных

In [ ]:
data_path = "../data_closed/"

## train data

### Чтение 

In [ ]:
df_train = pq.read_table(data_path + "train_data_10_10_24_10_11_24_final.parquet")

### Посмотрим, что мы загрузили

In [ ]:
df_train

Похоже на таблицу взаимодействия пользователей и товаров (interactions table)

Преобразуем в более удобный формат для просмотра данных (pandas)

In [ ]:
interactions_df = pd.DataFrame(df_train.to_pandas())
display(interactions_df)
display(interactions_df.dtypes)

Да, это таблица взаимодействий пользователей с айтемами. 

> ### Комментарий
> По одной одной этой таблице уже можно выдать рекомендации по взаимодействиям (пригодность данных &mdash; approved ✅)

Отметим, что subject_id похоже один и тот же для всех строк &mdash; проверим это: 

In [ ]:
interactions_df["subject_id"].unique()

Сократим размерность, дропнув столбец subject_id (константное значение)

In [ ]:
interactions_df = interactions_df.drop(columns=["subject_id"])

Следовательно, можно заключить, что номер "69020" &mdash; это внутренняя кодировка для категории товара. Теперь появилось представление за что отвечает каждое поле в таблице: 


|    Поле    |                      Значение                      |
| :--------: | :------------------------------------------------: |
| wbuser_id  |                  id пользователя                   |
|   nm_id    |                     id товара                      |
| subject_id |                id категории товара                 |
|     dt     | дата и время взаимодействия пользователя с товаром |
|    date    |     дата взаимодействия пользователя с товаром     |



Значит, будет строиться модель для предсказания взаимодейсвия пользователя с конкретным товаром.



Для удобства переименуем колонки
- `wbuser_id` $\rightarrow$ `user_id`
- `nm_id` $\rightarrow$ `item_id`

In [ ]:
# Переименовываем колонки
interactions_df = interactions_df.rename(
    columns={
        "wbuser_id": "user_id",
        "nm_id": "item_id",
    }
)

Посмотрим в принципе на количество уникальных значений для столбцов user_id, item_id, для столбцов отвечающих за дату и время посмотрим промежутки за которые предоставлены данные.  

In [ ]:
interactions_df["user_id"].unique().shape

~ 4 млн пользователей 

In [ ]:
interactions_df["item_id"].unique().shape

~ 400 тыс. товаров 

In [ ]:
print(f"min_date = {interactions_df['dt'].min()}")
print(f"max_date = {interactions_df['dt'].max()}")

Данные из датасета собраны за два дня.

Дропнем столбец date, т.к. он по сути дублирует столбец dt

In [ ]:
interactions_df = interactions_df.drop(columns=["date"])

### Посмотрим на полноту данных (наличие NaN значений в таблице):

In [ ]:
interactions_df.isnull().any()

Все поля таблицы заполнены.

### Добавим дополнительные параметры по взаимодействиям юзеров с товарами

Добавим количество взаимодействий пользователя с определенным товаром (`user_item_count`) и отношение этого количества к общему числу взаимодействий пользователя (получится рейтинг товара для пользователя `user_item_rating`) со всеми товарами за данный промежуток.
Так же добавим колонку с количеством интеракций товара среди всех позльователей (`item_count`) и отношение этого количества к числу всех интеракций (`item_rating`). 

> ### Комментарий
> Данные фитчи помогут проанализировать популярность товара для конекретного пользователя (локальный рейтинг) и составить общий рейтинг товара.
> Данные фитчи так же помогут в будущем при составлении ML-модели.

Так же дропнем столбец dt, т.к. он не существенен для дальнейшего анализа. 
Он потребуется в будущем, чтобы разделить данные при построении модели. 

In [ ]:
interactions_df = interactions_df.drop(columns=["dt"])

Теперь получится сократить размерность данных, т.к. существуют повторы взаимодействия пользователя с товарами: добавим новую колонку с количеством интеракций пользователя с конкретным товаром.

In [ ]:
interactions_df = (
    interactions_df.groupby(["user_id", "item_id"])
    .agg(
        {
            "item_id": "count",
        }
    )
    .rename(
        columns={
            "item_id": "user_item_count",
        }
    )
    .reset_index()
)
interactions_df

Составим user_item_rating, для этого посчитаем количество всех взаимодейстий пользователя

In [ ]:
total_users_interactions_count = (
    interactions_df[["user_id", "user_item_count"]]
    .groupby("user_id")
    .sum()
    .rename(
        columns={
            "user_item_count": "user_inter_count",
        }
    )
)
total_users_interactions_count

и разделим user_item_count на user_inter_count

In [ ]:
interactions_df = interactions_df.join(
    total_users_interactions_count,
    on="user_id",
    how="left",
)
interactions_df["user_item_rating"] = interactions_df["user_item_count"] / interactions_df["user_inter_count"]
interactions_df

Посчитаем количество взаимодействий с определенным товаром (`item_count`)

на его основе расчитаем рейтинг товара (`item_rating`)

In [ ]:
item_rating_df = (
    interactions_df[["item_id", "user_item_count"]]
    .groupby("item_id")
    .sum()
    .rename(
        columns={
            "user_item_count": "item_count",
        }
    )
)
item_rating_df["item_rating"] = (
    item_rating_df["item_count"] / item_rating_df.shape[0]
)
item_rating_df = item_rating_df.reset_index()

### Итого имеем две таблицы: 
1. Таблица взаимодействий пользователей с товарами, содержащая веса товаров для каждого отдельного пользователя
2. Таблица с общей популярностью (рейтингом) товаров

In [ ]:
display(interactions_df)
display(item_rating_df)

Теперь можно получить топ самых популярных товаров по каждом пользователю и топ самых популярных товаров в принципе. 

Если `user_item_count`, `user_item_rating` это фитчи относящиеся чисто к взаимодействию пользователей с товаром, то `item_rating` можно использовать как фитчу товара (по аналогии с рейтингом IMDB для фильмов).

> ### Комментарий
> Значения данных столбцов придется пересчитать, когда будет строится модель, т.к. там будет происходить разбиение данных на train, test по времени.


Отсортируем айтемы по популярности: 

In [ ]:
item_rating_df = item_rating_df.sort_values("item_rating", ascending=False)
item_rating_df

Теперь можно вывести топы популярных товаров

Топ 10 самых популярных товаров среди пользователей: 

In [ ]:
item_rating_df.head(10)

За два дня этими товарами взаимодействовали более 20 тыс. раз

### Вывод по данным train data

Из этого датасета удается получить качественнную таблицу взаимодействий пользователей с товарами. Также хорошо, что можно получить рейтинг товара (условный вес) как для всех пользователей, так и для каждого индивидуально. 

**Только по таблице взаимодействий уже можно нагенерить предсказания о следующих взаимодействиях пользователя с товарами.**

Сохраним таблицы в бинарник

In [ ]:
with open(data_path + "interactions.dill", "wb") as f:
    dill.dump(interactions_df, f)

with open(data_path + "item_rating.dill", "wb") as f:
    dill.dump(item_rating_df, f)

## text data

### Чтение данных

In [ ]:
df_text_pq = pq.read_table(data_path + "text_data_69020_final.parquet")

Возьмем только первый батч, чтобы посмотреть содержание таблицы 

> опытным путем было выяснено, что вся таблица в формате pandas не умещается в ОЗУ 

In [ ]:
df_text_pd = df_text_pq.to_batches()[0].to_pandas()

In [ ]:
df_text_pd

> Из поля `title` стало понятно, что `subject_id = 69020` из таблицы `train_data`, это кодировка для платьев.

Для данной таблицы имеем следующие поля

|      Поле       |       Значение        |
| :-------------: | :-------------------: |
|      title      |    название товара    |
|    brandname    |    название бренда    |
|      nm_id      |       id товара       |
| characteristics | характеристики товара |
|   description   |    описание товара    |



### Подробнее рассмотрим колонку chatacteristics

Подробнее рассмотрим характеристики товаров.
Выведем характеристики для 0, 10, 100 айтемов

In [ ]:
df_text_pd.iloc[0]["characteristics"], df_text_pd.iloc[10]["characteristics"], df_text_pd.iloc[100]["characteristics"]

> Это массивы соварей, следовательно, при составлении модели и подготовке данных значения из этих колонок придется парсить.

Характеристики товара &mdash; это признак, который отдается под заполние продавцу, поэтому признаков разное количество, а также разная полнота заполнения полей признаков.
 
Т.к., видимо, решается задача предсказания следующих просмотренных платьев, то можно углубиться в данную категорию товаров и выбрать общие признаки характеризующие данный тип изделий.

Попробуем взять условные 100 товаров и посмотреть какие признаки для них общие

In [ ]:
item_chars = {str.lower(i["charcName"]) for i in df_text_pd.iloc[0]["characteristics"]}

# Используем логику пересечения множеств
for i in range(1, 100):
    item_chars = item_chars & {
        str.lower(i["charcName"]) for i in df_text_pd.iloc[i]["characteristics"]
    }

item_chars

Видимо присутствуют товары, для которых поля с признаками не заполнены. 

Тогда попробуем определим признаки чуть-чуть подругому. Посмотрим пересечения множества признаков для 5 платьев, и отберем, как кажется, максимально логичные и понятные признаки для изделия

In [ ]:
item_chars = {str.lower(i["charcName"]) for i in df_text_pd.iloc[0]["characteristics"]}

for i in range(1, 5):
    item_chars = item_chars & {
        str.lower(i["charcName"]) for i in df_text_pd.iloc[i]["characteristics"]
    }

item_chars

В принципе следующего набора признаков должно быть достаточно, чтобы дать удовлетворительное описание товара

```python
{
    'длина юбки/платья',
    'модель платья',
    'назначение',
    'особенности модели',
    'покрой',
    'пол',
    'рисунок',
    'тип карманов',
    'тип ростовки',
    'тип рукава',
    'вид застежки',
    'вырез горловины',
    'страна производства',
}
```

если посмотреть, что предлагает каталог сайта WB, то там будут следующие характеристики товара (фильтры для категории `юбки\сарафаны`):

```python
{
    'Бренд',
    'С рейтингом от 4.5',
    'Рубли за отзыв',
    'Оригинальный товар',
    'Продавец',
    'Премиум-продавец',
    'Цвет',
    'Размер',
    'Состав',
    'Назначение',
    'Длина юбки/платья',
    'Покрой',
    'Страна производства',
    'Скидка',
}
```

Для выдачи более персонализированной рекомендации/подборки товаров нужно выбрать более полный список признаков.
Основываясь на приведенных данных, оставим следующие характреристики изделия, с которыми будем считаться


```python
{
    'длина юбки/платья',
    'модель платья',
    'назначение',
    'покрой',
    'рисунок',
    'тип карманов',
    'тип ростовки',
    'тип рукава',
    'вид застежки',
    'вырез горловины',
    'страна производства',
}
```



Для того, чтобы составить матрицы для обучения моделей необходимо знать какие данные имеются, что можно закодировать, и на какие категории можно подразделить товары. 
Из фильтров каталога WB можно вытащить основые значения для признаков: 

```python
{
    'Состав',
    'Назначение',
    'Длина юбки/платья',
    'Назначение',
    'Покрой',
    'Страна производства',
    'Цвет',
}
```
(Поместил таблицы со значениями для приведенных признаков в директории data/dress_chars/)

Остальные необходимо вытаскивать в ручную, итерируясь батчами по таблице.
> ### Вопрос
> Сколько нужно ОЗУ для хранения такой таблицы не в формате parquet? 

### Вывод по таблице text_data

Данная таблица дает признаки товаров, что хорошо для составления рекомендаций, а 
объединяя данную таблицу с топом популярных товаров можно извлечь:

- топ популярных брендов
- топ популярных покроев
- топ популярных длин

и т.п.

> ### Замечание 
> В топе популярных товаров отсутствуют некоторые позиции представленные в таблице text_data, для таких товаров можно будет поставить минимальный возмнжный рейтинг или вообще выбрать отрицательное значение

Резюмируя, можно сказать, что данные из этой таблицы пригодны для решения задачи (дополняют таблицу взаимодействий). 

## Images

В архиве images.zip хранятся изображения товаров.
- Название файла изображения соотносится с  `nm_id` товара.
- изображения размера 328x518 px

### Вывод по Images
Изображения пригодны для построения рекомендательной системы: 
можно представить в виде векторов в некотором пространстве и, рассчитывая их близость, выдавать кандидатов для ответа.